In [1]:
from bart_survival import surv_bart as sb
from bart_survival import simulation
import numpy as np

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
import importlib
importlib.reload(sb)

<module 'bart_survival.surv_bart' from '/home/twj8/projects/bs_final/BART-Survival/src/bart_survival/surv_bart.py'>

In [14]:
importlib.reload(sb)

<module 'bart_survival.surv_bart' from '/home/twj8/projects/bs_final/BART-Survival/src/bart_survival/surv_bart.py'>

In [51]:
rng = np.random.default_rng(1)
x_mat = simulation.get_x_matrix(
    N=100,
    x_vars=1,
    VAR_CLASS=[2],
    VAR_PROB=[.5],
    rng = rng
)
event_dict, sv_true, sv_scale_true = simulation.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(2 + .4*x_mat[:,0])",
    shape_f = "1",
    eos = 20,
    cens_scale=None,
    time_scale=5,
    true_only=False,
    rng = rng
)
# prepare data
# t_scale = sb.get_time_transform(event_dict["t_event"], time_scale = 5)
# y_sk = sb.get_y_sklearn(event_dict["status"], t_scale)
# trn = sb.get_surv_pre_train(y_sk, x_mat, weight=None)
trn = sb.get_surv_pre_train(
    y_time = event_dict["t_event"],
    y_status = event_dict["status"],
    x = x_mat,
    weight = None,
    time_scale = 5
)
            #  x_mat, weight=None)
# post_test = sb.get_posterior_test(y_sk=y_sk, x_test = x_mat)

mean shape 1.0
mean scale 9.315139848237454
mean time draws 8.513352463409824
4.0


In [59]:
importlib.reload(sb)

trn = sb.get_surv_pre_train(
    y_time = event_dict["t_event"],
    y_status = event_dict["status"],
    x = x_mat,
    weight = None,
    time_scale = 5
)

post_test = sb.get_posterior_test(
    y_time= event_dict["t_event"], 
    y_status = event_dict["status"], 
    x = x_mat,
    time_scale = 5
    )

In [60]:
# post_teste

{'post_x': array([[1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [1., 0.],
        [2., 0.],
        [3., 0.],
        [4., 0.],
        [1., 1.],
        [2., 1.],
        [3., 1.],


In [50]:
for i in range(20): #range(trn["y"].shape[0]):
    print("status: ",trn["y"][i,0], "time:", trn["x"][i,:])

status:  0.0 time: [1. 1.]
status:  0.0 time: [2. 1.]
status:  0.0 time: [3. 1.]
status:  0.0 time: [4. 1.]
status:  1.0 time: [5. 1.]
status:  0.0 time: [1. 1.]
status:  0.0 time: [2. 1.]
status:  0.0 time: [3. 1.]
status:  0.0 time: [4. 1.]
status:  0.0 time: [5. 1.]
status:  0.0 time: [6. 1.]
status:  0.0 time: [7. 1.]
status:  0.0 time: [8. 1.]
status:  0.0 time: [9. 1.]
status:  1.0 time: [10.  1.]
status:  0.0 time: [1. 0.]
status:  1.0 time: [2. 0.]
status:  0.0 time: [1. 1.]
status:  0.0 time: [2. 1.]
status:  0.0 time: [3. 1.]


In [6]:

SPLIT_RULES =  [
    "pmb.ContinuousSplitRule()", # time
    "pmb.OneHotSplitRule", # ccsr_ind_p2
]
model_dict = {"trees": 50,
    "split_rules": SPLIT_RULES
}
sampler_dict = {
            "draws": 10,
            "tune": 10,
            "cores": 2,
            "chains": 2,
            "compute_convergence_checks": False
        }

BSM = sb.BartSurvModel(model_config=model_dict, sampler_config=sampler_dict)

BSM.fit(
    y =  trn["y"],
    X = trn["x"],
    weights=trn["w"],
    coords = trn["coord"],
    random_seed=5
) 

/home/twj8/projects/bs_final/BART-Survival/.venv/lib/python3.10/site-packages/pymc/model/core.py:988: FutureWarning: Coords are now always mutable. Specifying `mutable` will raise an error in a future release
  warnings.warn(
/home/twj8/projects/bs_final/BART-Survival/.venv/lib/python3.10/site-packages/pymc/data.py:304: FutureWarning: MutableData is deprecated. All Data variables are now mutable. Use Data instead.
  warnings.warn(
Only 10 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Multiprocess sampling (2 chains in 2 jobs)
PGBART: [f]


Output()

Sampling 2 chains for 10 tune and 10 draw iterations (20 + 20 draws total) took 5 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data
	> predictor_names
	> offset

In [9]:

BSM2 = sb.BartSurvModel(model_config=model_dict, sampler_config=sampler_dict)

BSM2.fit(
    y =  trn["y"],
    X = trn["x"],
    weights=trn["w"],
    coords = trn["coord"],
    random_seed=5
) 



/home/twj8/projects/bs_final/BART-Survival/.venv/lib/python3.10/site-packages/pymc/model/core.py:988: FutureWarning: Coords are now always mutable. Specifying `mutable` will raise an error in a future release
  warnings.warn(
/home/twj8/projects/bs_final/BART-Survival/.venv/lib/python3.10/site-packages/pymc/data.py:304: FutureWarning: MutableData is deprecated. All Data variables are now mutable. Use Data instead.
  warnings.warn(
Only 10 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Multiprocess sampling (2 chains in 2 jobs)
PGBART: [f]


Output()

Sampling 2 chains for 10 tune and 10 draw iterations (20 + 20 draws total) took 3 seconds.


Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data
	> fit_data
	> predictor_names
	> offset

In [10]:
del(BSM)

In [12]:
import multiprocessing as mp 

childs = mp.active_children()

In [13]:
childs

[<ForkProcess name='SyncManager-1' pid=14783 parent=12971 started>,
 <ForkProcess name='SyncManager-4' pid=14848 parent=12971 started>]